In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [2]:
# portfolio of stocks
stocks = ['LULU', 'PYPL', 'ADBE', 'FTNT', 'CL', 'ZTS', 'LDOS', 'MNST', 'ADSK', 'MDLZ', 'ISRG', 'MRK', 'NVO', 'UNH', '4528.T', '4568.T', 'BRO', 'DEMANT.CO', 'HRMY', 'HSTM']

#### Download historical data from yf API

In [3]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("stocks.csv")

[*********************100%***********************]  20 of 20 completed


#### Download analysis from csv downloaded data

In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker       DEMANT.CO                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2020-01-02  209.800003  211.600006  208.199997  208.600006  223696.0   
2020-01-03  208.500000  208.800003  205.199997  206.600006  324721.0   
2020-01-06  206.000000  206.600006  187.000000  206.500000  355143.0   
2020-01-07  207.600006  208.899994  203.899994  204.399994  229116.0   
2020-01-08  203.899994  206.800003  202.199997  205.600006  348069.0   

Ticker           MDLZ                                              ...  \
Price            Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2020-01-02  47.902124  48.154421  47.075624  47.267025  7272900.0  ...   
2020-01-03  46.979935  47.475834  46.945134  47.188736  6791700.0  ...   
2020-01-06  47.319237  47.397537  46.849437  47.110439  5906400.0  ...   
2020-01-07  47.023440  47.049542  46.605841  46.910339  6716300.0  ...   
2020-01-08  47.006029  47.345328  46.901631  47.023430  5435400.0  ...   

Ticker            BRO                                              \
Price            Open       High        Low      Close     Volume   
Date                                                                
2020-01-02  38.076938  38.124990  37.673298  38.067329  1354300.0   
2020-01-03  37.673288  38.173036  37.644454  38.105762   910800.0   
2020-01-06  37.894332  38.153817  37.730951  38.076931  1476300.0   
2020-01-07  38.115377  38.269146  37.961609  37.961609   782400.0   
2020-01-08  38.038502  38.490194  37.990446  38.240322  1767500.0   

Ticker           4568.T                                                    
Price              Open         High          Low        Close     Volume  
Date                                                                       
2020-01-02          NaN          NaN          NaN          NaN        NaN  
2020-01-03          NaN          NaN          NaN          NaN        NaN  
2020-01-06  2231.429166  2247.341203  2181.196974  2195.861084  5632800.0  
2020-01-07  2205.845150  2227.373200  2191.805118  2194.925049  5121900.0  
2020-01-08  2206.469112  2220.821298  2164.349013  2191.181152  5244300.0  

[5 rows x 100 columns]

In [5]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [6]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()